# Deep Belief Networks  (DBN)

https://github.com/albertbup/deep-belief-network

Deep Belief Networks is a generative graphical model or class of deep neural network that control of multiple layers of hidden layers with connections between layers but not between units within each of the layer (https://en.wikipedia.org/wiki/Deep_belief_network).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2019-01-01'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,3.85,3.98,3.84,3.95,3.95,20548400
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700


In [3]:
dataset['Open_Close'] = (dataset['Open'] - dataset['Adj Close'])/dataset['Open']
dataset['High_Low'] = (dataset['High'] - dataset['Low'])/dataset['Low']
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Open_Close,High_Low,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,,,
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,-0.005025,0.030928,1,1,1,0.012658
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,-0.029925,0.047619,1,1,1,0.032500
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0.002387,0.034063,0,1,0,0.012107
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0.011820,0.028986,0,0,0,0.000000
2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0.026190,0.044444,0,0,1,-0.021531


In [4]:
X = np.array(dataset.drop(["Increase_Decrease"], axis=1))

Y = np.array(dataset["Increase_Decrease"])

In [5]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X = ss.fit_transform(X)

In [6]:
import tensorflow as tf

In [7]:
from dbn.tensorflow import SupervisedDBNClassification
# from dbn import SupervisedDBNClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [9]:
classifier = SupervisedDBNClassification(hidden_layers_structure = [256, 256],learning_rate_rbm=0.05, learning_rate=0.1, n_epochs_rbm=10,
                                        n_iter_backprop=100, batch_size=32, activation_function='relu', dropout_p=0.2)

In [10]:
classifier.fit(X_train, Y_train)

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 6.861141
>> Epoch 2 finished 	RBM Reconstruction error 17.113402
>> Epoch 3 finished 	RBM Reconstruction error 52.335089
>> Epoch 4 finished 	RBM Reconstruction error 210.406320
>> Epoch 5 finished 	RBM Reconstruction error 577.301492
>> Epoch 6 finished 	RBM Reconstruction error 736.563120
>> Epoch 7 finished 	RBM Reconstruction error 1240.883380
>> Epoch 8 finished 	RBM Reconstruction error 1808.350122
>> Epoch 9 finished 	RBM Reconstruction error 2855.419626
>> Epoch 10 finished 	RBM Reconstruction error 3895.758755
>> Epoch 1 finished 	RBM Reconstruction error 956991406080.000000
>> Epoch 2 finished 	RBM Reconstruction error 1497898024960.000000
>> Epoch 3 finished 	RBM Reconstruction error 1896433057792.000000
>> Epoch 4 finished 	RBM Reconstruction error 1299819659264.000000
>> Epoch 5 finished 	RBM Reconstruction error 1867366268928.000000
>> Epoch 6 finished 	RBM Reconstruction error 929195032576.000000
>>

SupervisedDBNClassification(batch_size=32, dropout_p=0.2,
              idx_to_label_map={0: 0, 1: 1}, l2_regularization=1.0,
              label_to_idx_map={0: 0, 1: 1}, learning_rate=0.1,
              n_iter_backprop=100, verbose=True)

In [11]:
Y_pred = classifier.predict(X_test)
print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))

Done.
Accuracy: 0.519841
